In [ ]:
!pip install astropy>=3.1
!pip install sunpy
!pip install imgaug==0.2.5
#!pip install stumpy
!pip install scipy --upgrade
#restart runtime after installation, it should work then
import pandas as pd
#import stumpy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib.patches import Rectangle
import datetime as dt
import scipy
import sklearn
import copy

plt.style.use('https://raw.githubusercontent.com/TDAmeritrade/stumpy/main/docs/stumpy.mplstyle')

from google.colab import drive
drive.mount('/content/drive')
DATASET_PATH = '/content/drive/MyDrive/Project/AzureDataset_FunctionCalls/'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.0 MB 3.9 MB/s 
     |████████████████████████████████| 1.1 MB 40.2 MB/s 
     |████████████████████████████████| 144 kB 48.2 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 271 kB 69.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 562 kB 4.2 MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.5-py3-none-any.whl size=561438 sha256=fe4accedd5ec3b62d9cc26c516d19b057efad87ab168b41b6f00d0ac1febd51d
  Stored in directory: /root/.cache/pip/wheels/60/dd/38/d1dc2cad2b6a66dc0249261004990bccb0f27985c74ba26e49
Successfully built imgaug
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
Looking in indexes: ht

In [ ]:
import time
start = 0*1440
n = start+1440
m=1

multiSeriesMain = pd.read_csv(DATASET_PATH+'4dadbc9a741ff86462d015981149d52bfcac6c1780e7ed6cd36bbd2664f398d0'+'.csv')
multiSeriesMain = multiSeriesMain.loc[:, ~multiSeriesMain.columns.str.contains('^Unnamed')].astype(float).reset_index(drop=True)
multiSeries = multiSeriesMain[start:n]

In [ ]:
#Euclidean Distance brute forcing
def BruteEuclidean(shapelets, timeSeries):
  distances = []
  for s in shapelets:
    point1 = np.array(s)
    temp = []
    for i in range(len(timeSeries)-m):
      point2 = timeSeries.iloc[i:i+m, :].to_numpy().T
      euclideanDist = np.linalg.norm(point1 - point2)
      if(euclideanDist == 0):
        euclideanDist = 99999999999999
      temp.append(euclideanDist)
    distances.append(temp)
  return distances

In [ ]:
#Calculate Internal Coverage
def calcCoverage(timeSeries, motifs):
  flatMotifs = []
  coveredIndices = []
  for l in motifs:
    for t in l:
      flatMotifs.append(t)
  for f in flatMotifs:
    for i in range(m):
      coveredIndices.append(f+i)
  return len(list(set(coveredIndices)))/len(timeSeries)

In [ ]:
#Shapelet Generator
#Creates mean shapelet from groups
def ShapeletGen(timeSeries, groupingIndices):
  shapelet = []
  dimensions = len(timeSeries.T)
  for d in range(dimensions):
    temp = []
    for i in range(m):
      temp.append(0)
    shapelet.append(temp)

  for g in groupingIndices:
    for d in range(dimensions):
      for i in range(m):
        shapelet[d][i] = shapelet[d][i] + timeSeries.iloc[g+i,d]

  for d in range(dimensions):
    for i in range(m):
      shapelet[d][i] = shapelet[d][i]/len(groupingIndices)
  return shapelet

In [ ]:
#Self Similar Euclidean Distance Calculations
windows = []
for i in range(len(multiSeries)-m):
  windows.append(multiSeries.iloc[i:i+m].T)
distances = BruteEuclidean(windows, multiSeries)

In [ ]:
avgDist = []
for d in distances:
  avgDist.append(np.mean(d))
print(np.mean(avgDist))

458535939912.6005


In [ ]:
#Generate Motifs
motifs = []
count = 0
for d in distances:
  temp2 = []
  temp2.append(count)
  for dd in range(len(d)):
    if d[dd] < 2000:
      temp2.append(dd)
  motifs.append(temp2)
  count = count + 1
#Filter and combine motifs
newMotifs = []
for i in range(len(motifs)):
  temp = []
  for t in motifs:
      if i in t:
        for v in t:
          temp.append(v)
  if len(temp) > 1:
    newMotifs.append(list(set(temp)))
print(len(newMotifs))
size = 0
count = 0
for t in newMotifs:
  if len(t) > size:
    size = len(t)
    result = t
  count = count + 1
print(result, size)

1439
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220,

In [ ]:
#Check if shapelet generated matches with self similar
shapelet = ShapeletGen(multiSeries, result)
shapeletDistances = BruteEuclidean([shapelet], multiSeries)
print(min(shapeletDistances[0]))
for i in range(len(shapeletDistances[0])):
  if(shapeletDistances[0][i] == min(shapeletDistances[0])):
    print(i)
print('Internal Coverage: ', calcCoverage(multiSeries, newMotifs))

1.5602419653256538
491
Internal Coverage:  0.9993055555555556


In [ ]:
shapelets = []
for n in newMotifs:
  shapelets.append(ShapeletGen(multiSeries, n))

In [ ]:
#Generate Motifs
motifs = []
count = 0
for d in distances:
  temp2 = []
  #temp2.append(count)
  for dd in range(len(d)):
    if d[dd] < 2000:
      temp2.append(dd)
  motifs.append(temp2)
  count = count + 1
#Filter and combine motifs
newMotifs = []
for i in range(len(motifs)):
  temp = []
  for t in motifs:
      if i in t:
        for v in t:
          temp.append(v)
  if len(temp) > 1:
    newMotifs.append(list(set(temp)))
print(len(newMotifs))
size = 0
count = 0
for t in newMotifs:
  if len(t) > size:
    size = len(t)
    result = t
  count = count + 1
print(result, size)

1439
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220,

In [ ]:
#Check if shapelet generated matches with external
print(len(result))
multiSeriesOther = multiSeriesMain[1440:2880]
shapelet = ShapeletGen(multiSeries, result)
shapeletDistances = BruteEuclidean([shapelet], multiSeries)
print(min(shapeletDistances[0]))
for i in range(len(shapeletDistances[0])):
  if(shapeletDistances[0][i] == min(shapeletDistances[0])):
    print(i)
print('External Coverage: ', calcCoverage(multiSeriesOther, newMotifs))

1439
1.5602419653256538
491
External Coverage:  0.9993055555555556
